pyTorch Wordle

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generate Dataset

In [2]:
class wordleDataset(Dataset):
    def __init__(self,data_dir,targetToClass,transform=None):
        self.data = []
        with open(data_dir,'r') as f:
            for line in f:
                lineData = line.split()
                target = lineData[0]
                guesses = lineData[1:len(lineData)]

                #Tokenize the inputs and convert to tensors
                
                target = targetToClass[target]
                for i in range(len(guesses)):
                    guesses[i] = targetToClass[guesses[i]]
                targetTensor = torch.tensor(target)
                guessTensor = torch.tensor(guesses)
                self.data.append([targetTensor,guessTensor])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [3]:
wordToNum = {}
numToWord = {}
index = 0
with open("words.txt",'r') as f:
    for line in f:
        lineData = line.split()
        numToWord.update({index:lineData[0]})
        wordToNum.update({lineData[0]:index})
        index += 1

wordleSet = wordleDataset("TrainingData.txt",wordToNum)


In [4]:
dataloader = DataLoader(wordleSet, batch_size=32, shuffle=True)
for outputData,inputData in dataloader:
    break

Training

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class neuralNetwork(nn.Module):
    def __init__(self,num_in,h1,h2,num_out):
        super().__init__()
        self.fc1 = nn.Linear(num_in,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2,num_out)


    def forward(self,x):
        #Inputs into fc1 through relu function (f(x) = x if x>0,else 0)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.out(x)
        
        return x

In [13]:
torch.manual_seed(234524)

model = neuralNetwork(10,256,256,15099)
model.to(device)
print(device)

cuda


In [8]:
critereon = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochNum = 100
losses,valLosses = [],[]

print(len(dataloader.dataset))

for i in range(epochNum):
    print(i,epochNum)
    
    model.train()
    curLoss = 0
    for output,input in dataloader:
        output,input = output.to(device), input.to(device)
        optimizer.zero_grad()
        input = input.float()

        modelOutputs = model(input)
        loss = critereon(modelOutputs,output)
        loss.backward()
        optimizer.step()
        curLoss += loss.item()*len(input)
    totalLoss = curLoss / len(dataloader.dataset)
    losses.append(totalLoss)

    #Evaluation
    model.eval()
    curLoss=0
    with torch.no_grad():
        for output,input in dataloader:
            output,input = output.to(device), input.to(device)
            input = input.float()

            modelOutputs = model(input)
            loss = critereon(modelOutputs,output)
            curLoss += loss.item() * len(input)
    valLoss = curLoss / len(dataloader.dataset)
    valLosses.append(curLoss)

    print(totalLoss)


625000
0 100
1508.43963495625
1 100
1508.4396343953124
2 100
1508.4396345921875
3 100
1508.439634425
4 100
1508.4396350359375
5 100
1508.43963375
6 100
